# `Parse JSON` : Car Dealer File

# <font color=red>Mr Fugu Data Science</font>

# (◕‿◕✿)

# Purpose & Outcome

+ Parse JSON Data

+ Establish a Primary key, split file into 2 tables based on key

`Consider these videos a way to look at different problems; inspire and create`

In [1]:
import pandas as pd
import json, itertools,re,os
from collections import defaultdict

# If you have multiple files we need to establish a way to make this coherent

+ Dig in with `os module` so we can find that nifty ole file

`Also, if you have a file in that directory but surrounded by many folders AKA sub'directories' then you will need to use a separate function. of which I have made in a prior video`

[File for that function if needed](https://github.com/MrFuguDataScience/SendDataFrom-R-Python-to-SQL-PSQL-Mongo/blob/master/HOW%20TO%20SEND%20DATA%20from%20Python%20to%20PSQL.ipynb) `please make sure to pay attention and modify for your needs/application`

In [25]:
rootdir = os.getcwd() # put your directory if you know
regex = re.compile(r'(VehicleFeaturesJSON.*$)')
my_list=[]
for root, dirs, files in os.walk(rootdir):
    for file in files:
        if regex.match(file):
            my_list.append(file)
my_list

['VehicleFeaturesJSON102280487.json',
 'VehicleFeaturesJSON102298407.json',
 'VehicleFeaturesJSON102633593.json']

# Load all files together as list of lists of python dictionaries

In [4]:
# my_list=['VehicleFeaturesJSON102280487.json',
#          'VehicleFeaturesJSON102298407.json','VehicleFeaturesJSON102633593.json']

f=[]
for i in my_list:
    with open(i,'r') as js:
        f.append([json.load(js)]) # store each json file as a list

f[0]

[{'warning_count': 0,
  'error_count': 0,
  'message_list': [],
  'token_expiration_minutes': 0,
  'used_vehicles': {'used_vehicle_list': [{'publish_date': '8/28/2020',
     'data_freq': 'D',
     'state': 'NT',
     'country': 'US',
     'vin': '102280487',
     'uvc': '2019485072',
     'groupnum': '4107',
     'model_year': '2019',
     'make': 'Kia',
     'model': 'Soul',
     'series': '+',
     'style': '4D Wagon',
     'mileage_cat': 'B',
     'class_code': 'A',
     'class_name': 'Small Car',
     'description_score': '',
     'first_values_flag': False,
     'risk_score': '',
     'region2': -50,
     'region3': 200,
     'region4': -100,
     'region5': 0,
     'region6': 0,
     'msrp': 20490,
     'retail_equipped': 21430,
     'price_includes': 'AT AC',
     'wheel_base': 101.2,
     'tire_size': '215/55R17',
     'gvw': 3232,
     'seat_cap': '5',
     'fuel_type': 'Gas',
     'fuel_cap': '14.2',
     'fuel_delivery': 'GDI',
     'hwy_mpg': '30',
     'city_mpg': '25',
  

# Enter nested json object 'aka' python dictionary

In [28]:
o=[]
for j in f:
#     print(j) # list of lists of dictionaries
    for i in j:
#         print(i) # enter each list
        for k in i['used_vehicles']['used_vehicle_list']: # enter each dictionary
            o.append(k)

df_=pd.DataFrame(o) # full data with nested column
df_

# at this point we have 3 documents and 54 columns before unnesting

,publish_date,data_freq,state,country,vin,uvc,groupnum,model_year,make,model,...,num_gears,ext_doors,airbags,anti_corrosion_warranty,basic_warranty,powertrain_warranty,road_assist_warranty,add_deduct_list,model_number_list,engine_turbocharged
0,8/28/2020,D,NT,US,102280487,2019485072,4107,2019,Kia,Soul,...,6,4,Side Curtain,"5-year/100,000-mile, Limited","5-year/60,000-mile, Limited","10-year/100,000-mile, Limited","5-year/60,000-mile","[{'uoc': '8C', 'name': '18-Inch Wheels', 'xcle...",[B2522],NaN
1,8/28/2020,D,NT,US,102298407,2017160869,0141,2017,Chevrolet,Malibu,...,6,4,Side Curtain,"6-year/100,000-mile","3-year/36,000-mile, Limited","5-year/60,000-miles, Limited","5-year/60,000-mile","[{'uoc': '8C', 'name': '18-Inch Wheels', 'xcle...",[1ZD69],Turbocharg
2,8/28/2020,D,NT,US,102633593,2017300901,0016,2017,Ford,Fusion,...,6,4,Side Curtain,5-year/Unlimited-mile,"3-year/36,000-mile","5-year/60,000-mile","5-year/60,000-mile","[{'uoc': '8C', 'name': '18-Inch Al Whl', 'xcle...",[P0H],NaN


# Think about how to separate into 2 tables with a common `Primary Key`

+ We are dealing with vehicles that in the real world would have a `vin number` which has telling signs.

`For example: you have a 17 digit/letter code representing your automobile.`

+ it will describe your airbag-type, country of origin, engine size, model year,trim level, etc.  

**`This could be a good start`**

https://www.edmunds.com/how-to/how-to-quickly-decode-your-vin.html



# `Trim Levels vs Model Number`:

+ A vehicle will have varying trim levels for any given vehicle model number


`-----------------------------------------`

**I will break this up and do it 2 different ways, you chose and look into what is more effective for you! (i.e. time vs memory management)**


# `First way`:

+ you need to expand the `'add_deduct_list'` column since it looks like this for each entry:

`0    [{'uoc': '8C', 'name': '18-Inch Wheels', 'xcle...
1    [{'uoc': '8C', 'name': '18-Inch Wheels', 'xcle...
2    [{'uoc': '8C', 'name': '18-Inch Al Whl', 'xcle...`


  
There is also something to note here; they are not all the same length. The key-value pairs align but they may repeat in different pairings for each car.

For example: 1 car may have 3 entries for this list of dictionaries while others have 2

`You will see shortly`

In [30]:
# Observe the repeating but for different items by 'name' (this is entry 1)

'''
except each 'name' corresponds to a separate item and that is what you are seeing
meaning that each items has all same keeps with different values unique to each "name"
''' 

df_['add_deduct_list'][0]



[{'uoc': '8C',
  'name': '18-Inch Wheels',
  'xclean': 350,
  'clean': 350,
  'avg': 350,
  'rough': 350,
  'auto': 'N',
  'resid12': 300,
  'resid24': 250,
  'resid30': 225,
  'resid36': 200,
  'resid42': 175,
  'resid48': 150,
  'resid60': 100,
  'resid72': 50},
 {'uoc': 'W2',
  'name': 'Primo Pkg',
  'xclean': 1600,
  'clean': 1600,
  'avg': 1600,
  'rough': 1600,
  'auto': 'N',
  'resid12': 1500,
  'resid24': 1400,
  'resid30': 1350,
  'resid36': 1300,
  'resid42': 1250,
  'resid48': 1200,
  'resid60': 1100,
  'resid72': 1000}]

**`First part below`**:
entering the DF column that is a list of dictionaries for each row (entry) and converting to tuples so I can do a dictionary with values as lists. To enter the DF I need to loop again using 'j'. To finally enter the dictionary I an going 1 level deeper with yet another loop 'k'. I have 2 places data is stored. 

'wow' will keep the default dictionary which will be key:column name, value as list. 

'b' is a list on the other hand to store a tuple with my index position as position 0 in tuple. I want this because I need to keep track of everything since each entry has different length due to the 'name' item that is unique for trim/model identification for the particular vehicle.

**`Second Step`**:

I iterate into 'b' because I am interested in the index position to do some aligning of variables for bookkeeping. This will allow me to take the 'vin','model' information with the new index and keep everything with corresponding rows for the new table 'DF' I will create. What is happening: we need a key for each table, but need to preserve the formatted new `'add_deduct_list'` expanded data which will have more rows than original data since we expanded.

In [34]:
# 
wow=defaultdict(list)
b=[]
for i in df_['add_deduct_list'].items():
    for j in i[1]:
        for k in j.items():
            wow[k[0]].append(k[1])  
        b.append([i,wow])
        
er=pd.DataFrame(wow)

# obtain the correct index for each entry that was expanded out
n=[]
for i in b:
    for j in i:
#         print(type(j[0]))
        if type(j[0])==int:
#         print(j)
            n.append(j[0]) # n=idx for each observation

new_df=pd.concat([pd.DataFrame(n),er],axis=1)

new_df=new_df.rename({0:'index_'},axis=1)
new_df


,index_,uoc,name,xclean,clean,avg,rough,auto,resid12,resid24,resid30,resid36,resid42,resid48,resid60,resid72
0,0,8C,18-Inch Wheels,350,350,350,350,N,300,250,225,200,175,150,100,50
1,0,W2,Primo Pkg,1600,1600,1600,1600,N,1500,1400,1350,1300,1250,1200,1100,1000
2,1,8C,18-Inch Wheels,225,225,225,225,N,200,175,150,150,125,125,100,100
3,1,N5,Navigation System,500,500,500,500,N,450,400,375,350,325,300,250,200
4,1,12,Sunroof,600,600,600,600,N,550,500,475,450,425,400,350,300
5,1,20,Leather,650,650,650,650,N,600,550,525,500,475,450,400,350
6,2,8C,18-Inch Al Whl,325,325,325,325,N,300,275,250,250,225,225,200,175
7,2,2D,1.5L 4cyl Ecoboost,400,400,400,400,Y,350,300,275,250,225,200,150,100
8,2,N5,Navigation System,450,450,450,450,N,400,350,325,300,275,250,200,150
9,2,12,Power Moonroof,550,550,550,550,N,500,450,425,400,375,350,300,250


# Intermediate Table:

+ This table will be used to line up, vin and model year with the other data so I have two tables based on a Primary and Foreign Key

`The "idx" will allow us to join the  temp Df to our new table/DF from last step`

In [8]:

nm=pd.DataFrame([range(len(df_)),df_['vin'],df_['model_number_list']]).T

nm.columns=['idx','vin','model_number_list']
nm


,idx,vin,model_number_list
0,0,102280487,[B2522]
1,1,102298407,[1ZD69]
2,2,102633593,[P0H]


# `Think about how to separate into 2 tables with a common Primary Key`

+ We are dealing with vehicles that in the real world would have a `vin number` which has telling signs.

`For example: you have a 17 digit/letter code representing your automobile.`

+ it will describe your airbag-type, country of origin, engine size, model year,trim level, etc.  

**`This could be a good start`**

https://www.edmunds.com/how-to/how-to-quickly-decode-your-vin.html

# `Second way`: to break up the "add_deduct_list" column

In [43]:
# chain: takes many iteratbles and returns 1, we are getting the idx
# iterating the column as tuple of key:vals

temp_df = pd.DataFrame(list(itertools.chain(*[zip([key]*len(val), val) 
            for key, val in df_.add_deduct_list.iteritems()])),
     columns=['idx', 'add_deduct_list'])  
temp_df

,idx,add_deduct_list
0,0,"{'uoc': '8C', 'name': '18-Inch Wheels', 'xclea..."
1,0,"{'uoc': 'W2', 'name': 'Primo Pkg', 'xclean': 1..."
2,1,"{'uoc': '8C', 'name': '18-Inch Wheels', 'xclea..."
3,1,"{'uoc': 'N5', 'name': 'Navigation System', 'xc..."
4,1,"{'uoc': '12', 'name': 'Sunroof', 'xclean': 600..."
5,1,"{'uoc': '20', 'name': 'Leather', 'xclean': 650..."
6,2,"{'uoc': '8C', 'name': '18-Inch Al Whl', 'xclea..."
7,2,"{'uoc': '2D', 'name': '1.5L 4cyl Ecoboost', 'x..."
8,2,"{'uoc': 'N5', 'name': 'Navigation System', 'xc..."
9,2,"{'uoc': '12', 'name': 'Power Moonroof', 'xclea..."


In [47]:
# df_1=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in temp_df.items() ]))

# Expand the rows of dictionaries:
expanded_df=pd.json_normalize(json.loads(temp_df.to_json(orient="records")))
expanded_df

,idx,add_deduct_list.uoc,add_deduct_list.name,add_deduct_list.xclean,add_deduct_list.clean,add_deduct_list.avg,add_deduct_list.rough,add_deduct_list.auto,add_deduct_list.resid12,add_deduct_list.resid24,add_deduct_list.resid30,add_deduct_list.resid36,add_deduct_list.resid42,add_deduct_list.resid48,add_deduct_list.resid60,add_deduct_list.resid72
0,0,8C,18-Inch Wheels,350,350,350,350,N,300,250,225,200,175,150,100,50
1,0,W2,Primo Pkg,1600,1600,1600,1600,N,1500,1400,1350,1300,1250,1200,1100,1000
2,1,8C,18-Inch Wheels,225,225,225,225,N,200,175,150,150,125,125,100,100
3,1,N5,Navigation System,500,500,500,500,N,450,400,375,350,325,300,250,200
4,1,12,Sunroof,600,600,600,600,N,550,500,475,450,425,400,350,300
5,1,20,Leather,650,650,650,650,N,600,550,525,500,475,450,400,350
6,2,8C,18-Inch Al Whl,325,325,325,325,N,300,275,250,250,225,225,200,175
7,2,2D,1.5L 4cyl Ecoboost,400,400,400,400,Y,350,300,275,250,225,200,150,100
8,2,N5,Navigation System,450,450,450,450,N,400,350,325,300,275,250,200,150
9,2,12,Power Moonroof,550,550,550,550,N,500,450,425,400,375,350,300,250


# Taking the Temp Table and expanded 'add_deduct_list':

+ We are achieving a new dataframe with the vin, mondel_number matching the rows of the expanded `'add_deduct_list'`

If you were to use a database to handle this *psql,sql* etc you will have your primary-key and foreign-key

In [11]:
# joining based on 'idx' 
add_deduct_list_df=pd.merge(nm,expanded_df,on='idx',how='outer')
add_deduct_list_df

,idx,vin,model_number_list,add_deduct_list.uoc,add_deduct_list.name,add_deduct_list.xclean,add_deduct_list.clean,add_deduct_list.avg,add_deduct_list.rough,add_deduct_list.auto,add_deduct_list.resid12,add_deduct_list.resid24,add_deduct_list.resid30,add_deduct_list.resid36,add_deduct_list.resid42,add_deduct_list.resid48,add_deduct_list.resid60,add_deduct_list.resid72
0,0,102280487,[B2522],8C,18-Inch Wheels,350,350,350,350,N,300,250,225,200,175,150,100,50
1,0,102280487,[B2522],W2,Primo Pkg,1600,1600,1600,1600,N,1500,1400,1350,1300,1250,1200,1100,1000
2,1,102298407,[1ZD69],8C,18-Inch Wheels,225,225,225,225,N,200,175,150,150,125,125,100,100
3,1,102298407,[1ZD69],N5,Navigation System,500,500,500,500,N,450,400,375,350,325,300,250,200
4,1,102298407,[1ZD69],12,Sunroof,600,600,600,600,N,550,500,475,450,425,400,350,300
5,1,102298407,[1ZD69],20,Leather,650,650,650,650,N,600,550,525,500,475,450,400,350
6,2,102633593,[P0H],8C,18-Inch Al Whl,325,325,325,325,N,300,275,250,250,225,225,200,175
7,2,102633593,[P0H],2D,1.5L 4cyl Ecoboost,400,400,400,400,Y,350,300,275,250,225,200,150,100
8,2,102633593,[P0H],N5,Navigation System,450,450,450,450,N,400,350,325,300,275,250,200,150
9,2,102633593,[P0H],12,Power Moonroof,550,550,550,550,N,500,450,425,400,375,350,300,250


# Now put let's join the two tables AKA DF's to show it works.

+ I didn't remove the original column on purpose because that is for the end user. 

In [24]:
# now everything lines up with 'vin' as our p-key if thinking of DB usage later
pd.merge(add_deduct_list_df,df_,on='vin',how='inner')

,idx,vin,model_number_list_x,add_deduct_list.uoc,add_deduct_list.name,add_deduct_list.xclean,add_deduct_list.clean,add_deduct_list.avg,add_deduct_list.rough,add_deduct_list.auto,...,num_gears,ext_doors,airbags,anti_corrosion_warranty,basic_warranty,powertrain_warranty,road_assist_warranty,add_deduct_list,model_number_list_y,engine_turbocharged
0,0,102280487,[B2522],8C,18-Inch Wheels,350,350,350,350,N,...,6,4,Side Curtain,"5-year/100,000-mile, Limited","5-year/60,000-mile, Limited","10-year/100,000-mile, Limited","5-year/60,000-mile","[{'uoc': '8C', 'name': '18-Inch Wheels', 'xcle...",[B2522],NaN
1,0,102280487,[B2522],W2,Primo Pkg,1600,1600,1600,1600,N,...,6,4,Side Curtain,"5-year/100,000-mile, Limited","5-year/60,000-mile, Limited","10-year/100,000-mile, Limited","5-year/60,000-mile","[{'uoc': '8C', 'name': '18-Inch Wheels', 'xcle...",[B2522],NaN
2,1,102298407,[1ZD69],8C,18-Inch Wheels,225,225,225,225,N,...,6,4,Side Curtain,"6-year/100,000-mile","3-year/36,000-mile, Limited","5-year/60,000-miles, Limited","5-year/60,000-mile","[{'uoc': '8C', 'name': '18-Inch Wheels', 'xcle...",[1ZD69],Turbocharg
3,1,102298407,[1ZD69],N5,Navigation System,500,500,500,500,N,...,6,4,Side Curtain,"6-year/100,000-mile","3-year/36,000-mile, Limited","5-year/60,000-miles, Limited","5-year/60,000-mile","[{'uoc': '8C', 'name': '18-Inch Wheels', 'xcle...",[1ZD69],Turbocharg
4,1,102298407,[1ZD69],12,Sunroof,600,600,600,600,N,...,6,4,Side Curtain,"6-year/100,000-mile","3-year/36,000-mile, Limited","5-year/60,000-miles, Limited","5-year/60,000-mile","[{'uoc': '8C', 'name': '18-Inch Wheels', 'xcle...",[1ZD69],Turbocharg
5,1,102298407,[1ZD69],20,Leather,650,650,650,650,N,...,6,4,Side Curtain,"6-year/100,000-mile","3-year/36,000-mile, Limited","5-year/60,000-miles, Limited","5-year/60,000-mile","[{'uoc': '8C', 'name': '18-Inch Wheels', 'xcle...",[1ZD69],Turbocharg
6,2,102633593,[P0H],8C,18-Inch Al Whl,325,325,325,325,N,...,6,4,Side Curtain,5-year/Unlimited-mile,"3-year/36,000-mile","5-year/60,000-mile","5-year/60,000-mile","[{'uoc': '8C', 'name': '18-Inch Al Whl', 'xcle...",[P0H],NaN
7,2,102633593,[P0H],2D,1.5L 4cyl Ecoboost,400,400,400,400,Y,...,6,4,Side Curtain,5-year/Unlimited-mile,"3-year/36,000-mile","5-year/60,000-mile","5-year/60,000-mile","[{'uoc': '8C', 'name': '18-Inch Al Whl', 'xcle...",[P0H],NaN
8,2,102633593,[P0H],N5,Navigation System,450,450,450,450,N,...,6,4,Side Curtain,5-year/Unlimited-mile,"3-year/36,000-mile","5-year/60,000-mile","5-year/60,000-mile","[{'uoc': '8C', 'name': '18-Inch Al Whl', 'xcle...",[P0H],NaN
9,2,102633593,[P0H],12,Power Moonroof,550,550,550,550,N,...,6,4,Side Curtain,5-year/Unlimited-mile,"3-year/36,000-mile","5-year/60,000-mile","5-year/60,000-mile","[{'uoc': '8C', 'name': '18-Inch Al Whl', 'xcle...",[P0H],NaN


# <font color=red>LIKE</font>, Share & 

# <font color=red>SUB</font>scribe

# Citations & Help:

# <font size=7>◔̯◔</font>

https://stackoverflow.com/questions/28655405/how-to-deal-with-pandas-column-that-has-a-list-of-dicts-in-every-cell

https://stackoverflow.com/questions/39293968/how-do-i-search-directories-and-find-files-that-match-regex